# Complete PH workflow

The **PH** pacakge has a very modular structure for dealing with the different parts of the **PH** benchmark test case:

* module **ansatzes** deals with ansatz circuit creation and solving (see notebook: **01_Ansatzes.ipynb**)
* module **parent_hamiltonian**: has functions and class for computing the Parent Hamiltonian and its Pauli String decomposition (see notebook **02_Using_PH_Class.ipynb**)
* module *vqe_step*: deals with the computation of the ground state energy for a given ansatz and parent hamiltonian (see notebook **03_ParentHamiltonian_execution.ipynb**).

In this notebook the module **workflow** is reviewed. This module group all the aforementioned modules for execute a complete Benchamrk Test Case for a **PH Kernel**


In [ ]:
import logging
logging.basicConfig(
    format='%(asctime)s-%(levelname)s: %(message)s',
    datefmt='%m/%d/%Y %I:%M:%S %p',
    level=logging.INFO
    #level=logging.DEBUG
)
logger = logging.getLogger('__name__')

In [ ]:
import numpy as np
import pandas as pd
import time

In [ ]:
import sys
sys.path.append("../")

In [ ]:
# myQLM qpus
from qat.qpus import PyLinalg, CLinalg
qpu_c = CLinalg()
qpu_p = PyLinalg()
# QLM qpus
from qlmaas.qpus import LinAlg, MPS
qpu_qaass = LinAlg()
qpu_mps = MPS()

In [ ]:
from workflow import workflow

In order to use the *workflow* function from **workflow** module a configuration dictionary should be provided. Following cell show a complete configuration

In [ ]:
nqubits = 8
depth = 4
i = 0
ansatz = ["simple01", "simple02", "lda", "hwe"][i]
if i== 0:
    t_inv = True
else:
    t_inv = False
truncation = None
nb_shots = 0
qpus = ["c", "python", "qlmass", "mps"]
qpu_ansatz = qpus[0]
qpu_ph = qpus[0]
save = True
folder = "Savings/"
configuration = {
    # ansatz related configuration
    "nqubits": nqubits,
    "depth": depth,
    "ansatz": ansatz,
    # parent hamiltonian configuration
    "t_inv": t_inv,
    # vqe step configuration
    "truncation": truncation,
    "nb_shots": nb_shots,
    "qpu_ansatz": qpu_ansatz,
    "qpu_ph": qpu_ph,
    "save": save,
    "folder": folder
    
}

In [ ]:
pdf = workflow(**configuration)

In [ ]:
pdf

We can truncate Pauli terms !!

In [ ]:
configuration.update({"truncation": 3})

In [ ]:
pdf = workflow(**configuration)

In [ ]:
pdf

In [ ]:
configuration.update({"truncation": 2})
pdf = workflow(**configuration)

In [ ]:
pdf

## Command Line

The **workflow** module can be executed from command line. Several arguments can be provided for configuring the complete execution. For getting a help type:

**python workflow.py -h**

The following argumnets can be provided:

* -nqubits NQUBITS      Number of qbits for the ansatz.
* -depth DEPTH          Depth for ansatz.
* -ansatz ANSATZ        Ansatz type: simple01, simple02, lda or hwe.
* -qpu_ansatz QPU_ANSATZ: QPU for ansatz simulation: [qlmass, python, c, mps]
* --t_inv               Setting translational invariant of the ansatz
* -truncation TRUNCATION: Truncation for Pauli coeficients.
* -nb_shots NB_SHOTS    Number of shots
* -qpu_ph QPU_PH        QPU for parent hamiltonian simulation: [qlmass, python, c]
* -folder FOLDER        Path for storing results
* --save                For storing results

## Masive Execution

As in the other cases we can execute masive executions of the **workflow** module. The mandatory files are:

* **workflow.json**: JSON file with the complete desired configuration for the workflow. Each posible configuration parameter in the JSON file has associated a list with the desired values. The total number of workflows will be all the posible combinations of the values of all the parameters. 
    * Example: if *nqubits: [10, 14]*, *depth: [1, 2, 3, 4]* and *ansatz: ["simple01", "simple02"]* (and the otheres parameters has only a one element) then the posible workflows will be 2 * 4 * 2= 16. 
* **launch_workflow.py** this script allows to configure the workflows taking the configuration from the **workflow.json** and executes them. Use **python launch_workflow -h** for getting a help. The following arguments can be provided:
    * --all: for selecting all the posible workflows resulting from **workflow.json** file.
    * -id ID: for selecting only one posible workflow (the number given by ID) from all the posible workflowss from **workflow.json** file.
    * --print: for printing the configuration of the workflow
    * --count: give the number of all the posible combinations resulting from **workflow.json** file.
    * --exe: for executing a complete workflow. If --all is provided all posible workflow combinations from **workflow.json** are executes. If -id ID is provided only the ID workflow will be executed.